<a href="https://colab.research.google.com/github/Da-Heon/Record_DataAnalysis/blob/main/%EA%B0%80%EC%84%A4_%EA%B2%80%EC%A0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/05_두잇데이터분석/mpg.csv')
df

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,category
0,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
1,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
2,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
3,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
4,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
229,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
230,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
231,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
232,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


## compact 자동차와 suv 자동차의 도시 연비 t검점

In [4]:
df.query('category in ["compact", "suv"]')\
    .groupby('category', as_index=False)\
    .agg(n=('category', 'count'), mean=('cty', 'mean'))

,category,n,mean
0,compact,47,20.12766
1,suv,62,13.50000


- 앞에서 구한 평균 차이가 통계적으로 유의한지 t 검정을 알아보자
- 먼저 category가 compact인 행과 suv인 행의 cty를 추출해 변수에 할당하자

In [5]:
compact = df.query('category == "compact"')['cty']
suv = df.query('category == "suv"')['cty']

In [7]:
from scipy import stats

stats.ttest_ind(compact, suv, equal_var = True)

TtestResult(statistic=np.float64(11.917282584324107), pvalue=np.float64(2.3909550904711282e-21), df=np.float64(107.0))

```
출력 결과에서 p-value가 유의확률을 의미합니다. 일반적으로 유의확률 5%를 판단 기준으로 삼고
p-value가 0.05 미만이면 '집단 간 차이가 통계적으로 유의하다'고 해석합니다.
실제로는 차이가 없는데 이런 정도의 차이가 우연히 관찰될 확률이 5%보다 작다면, 이 차이를 우연이라고 보기 어렵다고 결론 내리는 것입니다.
'2.3909550904711282e-21' 은 유의확률이 '2.390955' 앞에 0이 21개 있는 값 보다 작다는 의미입니다.
p-value가 0.05보다 작기 때문에 이 분석결과는 compact와 suv간 평균 도시 연비 차이가 통계적으로 유의하다고 결론 내릴 수 있습니다.
```

## 일반 휘발유와 고급 휘발유의 도시 연비 t검정

In [10]:
# 기술 통계 분석
df.query('fl in ["r", "p"]')\
    .groupby('fl', as_index=False)\
    .agg(n=('fl', 'count'), mean=('cty', 'mean'))

,fl,n,mean
0,p,52,17.365385
1,r,168,16.738095


In [11]:
regular = df.query('fl == "r"')['cty']
pre = df.query('fl == "p"')['cty']

In [13]:
# t-test
stats.ttest_ind(regular, pre, equal_var=True)

TtestResult(statistic=np.float64(-1.066182514588919), pvalue=np.float64(0.28752051088667036), df=np.float64(218.0))

```
p-value가 0.05보다 큰 0.2875.. 입니다. 실제로는 차이가 없는데 우연에 의해 이런 정도의 차이가 관찰될 확률이 28.75%라는 의미입니다.
따라서 일반 휘발유와 고급 휘발유를 사용하는 자동차의 도시 연비 차이가 통계적으로 유의하지 않다고 결론 내릴 수 있습니다.
고급 휘발유 자동차의 도시 연비 평균이 0.6 정도 높지만 이런 정도의 차이는 우연히 발생했을 가능성이 크다고 해석할 수 있습니다.
```